In [16]:
import pandas as pd
import numpy as np

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.metrics.pairwise import cosine_similarity

# Spotipy credentials
SPOTIPY_CLIENT_ID = 'f9808bd72ef14052871b700f4ea7c302'
SPOTIPY_CLIENT_SECRET = '085a308ed59e4baebf23cc85f8133723'

track_id = '6MteoNtr1TUcYJrIHsAntE' # Reverse Cowgirls - Hot Dinners

In [17]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
                                client_id=SPOTIPY_CLIENT_ID,
                                client_secret=SPOTIPY_CLIENT_SECRET))
# Get features for track
track_features = sp.audio_features(track_id)[0]
track_features

{'danceability': 0.231,
 'energy': 0.655,
 'key': 2,
 'loudness': -9.872,
 'mode': 1,
 'speechiness': 0.049,
 'acousticness': 0.0279,
 'instrumentalness': 0,
 'liveness': 0.136,
 'valence': 0.656,
 'tempo': 206.053,
 'type': 'audio_features',
 'id': '6MteoNtr1TUcYJrIHsAntE',
 'uri': 'spotify:track:6MteoNtr1TUcYJrIHsAntE',
 'track_href': 'https://api.spotify.com/v1/tracks/6MteoNtr1TUcYJrIHsAntE',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6MteoNtr1TUcYJrIHsAntE',
 'duration_ms': 206587,
 'time_signature': 4}

In [19]:
cols = ['energy', 'danceability',
        'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence']
track_data = [track_features[col] for col in cols]
track_data

[0.655, 0.231, 0.049, 0.0279, 0, 0.136, 0.656]

In [20]:
df = pd.read_csv('tracks.csv')
df.drop(columns=['duration_ms', 'explicit', 'key',
                 'popularity', 'mode', 'release_date',
                 'tempo'], inplace=True)
df.head()

,id,name,artists,id_artists,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,['Uli'],['45tIt06XoI0Iio4LBEVpls'],0.645,0.4450,-13.338,0.4510,0.674,0.7440,0.151,0.127,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],0.695,0.2630,-22.136,0.9570,0.797,0.0000,0.148,0.655,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0.434,0.1770,-21.180,0.0512,0.994,0.0218,0.212,0.457,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],0.321,0.0946,-27.961,0.0504,0.995,0.9180,0.104,0.397,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],0.402,0.1580,-16.900,0.0390,0.989,0.1300,0.311,0.196,4


In [21]:
# Create Item Profiles DataFrame
cols = ['energy', 'danceability',
        'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence']
item_profiles = df[cols]
item_profiles.head()

,energy,danceability,speechiness,acousticness,instrumentalness,liveness,valence
0,0.4450,0.645,0.4510,0.674,0.7440,0.151,0.127
1,0.2630,0.695,0.9570,0.797,0.0000,0.148,0.655
2,0.1770,0.434,0.0512,0.994,0.0218,0.212,0.457
3,0.0946,0.321,0.0504,0.995,0.9180,0.104,0.397
4,0.1580,0.402,0.0390,0.989,0.1300,0.311,0.196


In [22]:
labels = df[['id', 'artists', 'name']]
labels['artists'] = labels['artists'].str.replace(r'[^A-Za-z\s]+', '')
labels.head()

/home/stephen/Projects/django/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,artists,name
0,35iwgR4jXetI318WEWsa1Q,Uli,Carve
1,021ht4sdgPcrDgSk7JTbKY,Fernando Pessoa,Capítulo 2.16 - Banquero Anarquista
2,07A5yehtSnoedViJAZkNnc,Ignacio Corsini,Vivo para Quererte - Remasterizado
3,08FmqUhxtyLTn6pAh6bk45,Ignacio Corsini,El Prisionero - Remasterizado
4,08y9GfoqCWfOGsKdwojr5e,Dick Haymes,Lady of the Evening


In [23]:
def get_most_similar(item_profiles, user_profile, labels):
    """
    Uses Cos Theta to calculate similar items to user profile
    """
    sim_df = pd.DataFrame(data=labels, columns=['id', 'artists', 'name'])
    sim_df['similarity'] = cosine_similarity(item_profiles, user_profile)
    sim_df.sort_values(by=['similarity'], ascending=False, inplace=True)
    return sim_df

In [24]:
most_sim = get_most_similar(item_profiles.values, np.array(track_data).reshape(1, -1), labels)
most_sim.head(20)

,id,artists,name,similarity
489711,1UA1La09yOjJIJZg2a9rUl,El Guato,Las Piñatas,0.999925
315812,7xgPkZw5sOBBNHe8PhOg5l,Kr ldutnsskla,Það var einu sinni strákur,0.999640
86387,6W7BJWngFn9AEfZQrV73Ft,The Pogues,Boat Train - 2013 Mix,0.999461
479821,6REkhlp3lPDXlhyaPWRUby,Descendents,Descendents,0.999447
309793,6FUAlX42Hm3FHoU4EgiQ3F,Ibens,Jeg savner min blå cykel,0.999434
227590,11jNREQU3a7S0iGKm8gTjV,Splean,Весь этот бред,0.999430
168912,5YE4ef8FbD7RlkjoegKPLd,Klamydia,Töihin armeijaan,0.999309
578284,7dkf2ksoWC6PhhlzjtTRWE,Ebba Grn,Flyger,0.999281
559864,1zDPaZJxf4zbNPThFqjqu8,Takuro Yoshida,人生を語らず,0.999230
324662,2RcmIAhPnodF87lK3fJT6v,Status Quo,Technicolor Dreams - Mono Version,0.999229


In [25]:
sp.search('as gods my witness cowgirls', type='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=as+gods+my+witness+cowgirls&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6OLtkCTj8hGgKfK2hu22fN'},
       'href': 'https://api.spotify.com/v1/artists/6OLtkCTj8hGgKfK2hu22fN',
       'id': '6OLtkCTj8hGgKfK2hu22fN',
       'name': 'The Reverse Cowgirls',
       'type': 'artist',
       'uri': 'spotify:artist:6OLtkCTj8hGgKfK2hu22fN'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
   

In [26]:
hot_dinners_features = sp.audio_features('6MteoNtr1TUcYJrIHsAntE')
hot_dinners_features

[{'danceability': 0.231,
  'energy': 0.655,
  'key': 2,
  'loudness': -9.872,
  'mode': 1,
  'speechiness': 0.049,
  'acousticness': 0.0279,
  'instrumentalness': 0,
  'liveness': 0.136,
  'valence': 0.656,
  'tempo': 206.053,
  'type': 'audio_features',
  'id': '6MteoNtr1TUcYJrIHsAntE',
  'uri': 'spotify:track:6MteoNtr1TUcYJrIHsAntE',
  'track_href': 'https://api.spotify.com/v1/tracks/6MteoNtr1TUcYJrIHsAntE',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6MteoNtr1TUcYJrIHsAntE',
  'duration_ms': 206587,
  'time_signature': 4}]